In [ ]:
import pandas as pd

## Data types and summary statistics, continued

These data show daily weather conditions measured in New York City.

In [ ]:
weather = pd.read_csv('https://raw.githubusercontent.com/dlevine01/urban-data-analysis-course/refs/heads/main/Data/Source%20Data/weather_data_nyc_centralpark_2016.csv')

First, inspect this data a bit:
- what does each row represent?
- what does each column represent?

In [ ]:
weather.head()

What data types does it seem like each column should be?

Check that pandas read the correct data types:

In [ ]:
weather.dtypes

Uh oh! it looks like not.

Pandas will infer data types, and if it can't tell or if a column has mixed types, it will fall back on the catch-all 'object', which is strings or a mix of strings and numbers.

We can check why that column did not get parsed as numeric:

In [ ]:
(
    weather
    .sort_values('precipitation')
)

See something that is not a number?

Let's filter to see all the rows with letters where we expect numbers

In [ ]:
(
    weather
    [
        weather['precipitation'].str.isalpha()
    ]
)

If you found this dataset in the wild, this is where you would go check the data documentation to know what the "T" code means. I'll save you some work and tell you it means "none recorded"

So now we have a choice, do we want that to be None, or zero?

When computing a mean or a median, a None won't count, but a zero will.

This is not a trivial decision!

For now, we will treat the not-recoded values as None.

First, set the columns as numeric. If we don't tell pandas what to do with values it can't turn into numbers, it will raise an error: 

In [ ]:
pd.to_numeric(weather['precipitation'])

(If we had skipped the exploration above, this would be a good warning that something in this column is not what we expect)

If we tell pandas to 'coerce' the errors, it will replace values it can't turn to numbers to None values.

In [ ]:
pd.to_numeric(weather['precipitation'], errors='coerce')

_(An extra caution about coercing text to numbers: check for commas between thousands of large numbers. by default, `to_numeric` won't understand those, so you need to first strip the commas.)_

Assign this re-cast column to a new column.

In [ ]:
weather['precipitation_n'] = pd.to_numeric(weather['precipitation'],errors='coerce')

(We could also overwrite the existing column by assigning the transformation to the same name. You will often see this approach. But the downside is that it destroys your original data. If later on we find that we should have parsed this a different way, it's better if we still have the original data to refer to. Don't make invisible mistakes.)

Phew! Now our data is in the format we expect and we can start analyzing it.

### Summary statistics

How hot is the hottest day?

In [ ]:
weather['maximum temperature'].max()

What's the average temperature?

In [ ]:
weather['average temperature'].mean()

Is this about the same as the representative middle temperature?

In [ ]:
weather['average temperature'].median()

What does the difference tell you about the skew of the data?

What's the average rainfall?

In [ ]:
weather['precipitation_n'].mean()

how about the typical day rainfall?

In [ ]:
weather['precipitation_n'].median()

What does this difference tell you?

How many days is there any rainfall?

There's not a single built-in method for that like there is for `.mean()` or `.median()`, but you can string together a few methods:

` > 0` returns `True` if the value is greater than zero:

In [ ]:
weather['precipitation_n'] > 0

You can also use the syntax `.gt(0)`:

In [ ]:
weather['precipitation_n'].gt(0)

But you still want to condense this new column to a summary statistic. pandas counts `True` as 1 and `False` as 0, so the total of this column is the number of instances of `True`

In [ ]:
weather['precipitation_n'].gt(0).sum()

Because the mean is computed as the sum divided by the count, the mean of a boolean column like this is the portion of values that are `True` (or multiply this by 100 to get the percent that meet the condition)

In [ ]:
weather['precipitation_n'].gt(0).mean()

In [ ]:
weather['maximum temperature'].mean()

In [ ]:
(
    weather['average temperature']
    .gt(weather['maximum temperature'].min())
    .mean().sum()
)

## Selecting data from lists and dicts

Python uses square brackets (`[ ]`) to select items.

Items in lists (or tuples) can be selected by their _index_. 

Python is _zero-indexed_, meaning the first item is numbered zero.

In [ ]:
fruits = ['apple','banana','pineapple','mango','orange']

In [ ]:
fruits[0]

In [ ]:
fruits[1]

Negative indexers start counting from the end of the list (these start with -1 as the first item from the end)

In [ ]:
fruits[-1]

A range of values can be selected using a colon `:`. This selects all items _including_ the first index but _not including_ the last index:

In [ ]:
fruits[1:3]

Using a colon on just one side selects all values from or to that value:

In [ ]:
fruits[2:]

In [ ]:
fruits[:2]

In [ ]:
fruits[-2:]

Values can be assigned to a place in a list the same way:

In [ ]:
fruits[2] = 'strawberry'

In [ ]:
fruits

Lists can be extended by adding lists together:

In [ ]:
fruits + ['papaya']

Selecting from tuples works the same way:

In [ ]:
colors = ('red','blue','green')

In [ ]:
colors[1]

But tuples are immutable so values cannot be assigned and new values cannot be added.

Items can be selected from dicts by their _key_: 

In [ ]:
n_legs = {
    'dog' : 4,
    'pigeon' : 2,
    'spider' : 8,
    'goldfish' : 0
}

In [ ]:
n_legs['dog']

calling a key that does not exist will raise an error

In [ ]:
n_legs['cat']

alternatively, use the `.get()` method which will return the matching value, or None

In [ ]:
n_legs.get('dog')

In [ ]:
n_legs.get('cat')

## Inequalities (comparison tests)

Inequalities are tests used to compare values. They return a boolean, `True` or `False`.

In Python, "is equal to" is represented with two equals signs: `==`. (This is distinguished from a single equals sign used for assignment)

In [ ]:
4 == 4

In [ ]:
4 == 5

In [ ]:
my_number = 5   # this equals sign is used for assignment

my_number == 5   # these double equals signs are used for an equality test

text values can also be equal:

In [ ]:
'pizza' == 'pizza'

...but must be exact matches:

In [ ]:
'pizza' == 'Pizza'

"not equal to" is represented as `!=`. This returns exactly the opposite of `==`

In [ ]:
4 != 4

In [ ]:
4 != 5

Greater than (`>`), greater than or equal to (`>=`), less than (`<`), and less than or equal to (`<=`) can be used to compare numerical values:

In [ ]:
4 < 4

In [ ]:
4 <= 4

Inequalities can be logically combined with the keywords `and` or `or`

In [ ]:
my_number = 5

In [ ]:
my_number > 3 and my_number < 10

In [ ]:
my_number < 3 or my_number == 10

## Inequality comparisons in pandas

In [ ]:
trees = pd.read_csv(
    'https://data.cityofnewyork.us/api/views/hn5i-inap/rows.csv?accessType=DOWNLOAD',
    usecols=[
        'OBJECTID',
        'GenusSpecies',
        'DBH',
        'StumpDiameter',
        'TPStructure',
        'TPCondition',
        'Location',
        'PlantedDate'
    ]
)

Applying an inequality to a column of data returns a boolean column indicating whether each value does or does not pass the test. 

In [ ]:
trees['DBH']

In [ ]:
trees['DBH'] > 10

pandas also has an alternative syntax: `.gt()` for `>` (greater than), `.ge()` for `>=` (greater than or equal to), `.eq()` for `==` (equals), etc. 
They are exactly equivalent.

In [ ]:
trees['DBH'].gt(10)

Inequalities can also be used for str-type columns. Again, this will only return `True` for exact matches.

In [ ]:
trees['TPCondition']

In [ ]:
trees['TPCondition'].eq('Good')

Inequalities can be logically combined with `&` for and or `|` for or

In [ ]:
(
    trees['DBH'].ge(5)
    &
    trees['DBH'].lt(10)
)

(When using the `==`-type syntax, each condition needs to be wrapped in parentheses)

In [ ]:
(
    (trees['DBH'] >= 5)
    &
    (trees['DBH'] < 10)
)

Conditions across more than one column can be combined:

In [ ]:
(
    trees['DBH'].lt(1)
    |
    trees['TPCondition'].eq('Dead')
)

As series, these boolean columns are not that meaningful. But they can be summarized: 

`True` is counted as 1 and `False` as 0, so the `sum` of a boolean column is equal to the number of `True` values:

In [ ]:
(
    (
        trees['DBH'].lt(1)
        |
        trees['TPCondition'].eq('Dead')
    )
    .sum()
    # .item() # hide the data type and just show the value
)

The mean (the sum divided by the count) of a boolean column is the _proportion_ of values that are `True`.

In [ ]:
(
    (
        trees['DBH'].lt(1)
        |
        trees['TPCondition'].eq('Dead')
    )
    .mean()
    # .item() # hide the data type and just show the value
)

Additionally, we will use these tests to _filter_ data.

## Filtering and selecting rows

We've seen that we can select columns by name (equivilent to selecting values from a dict)

In [ ]:
trees['DBH']

We can select rows or columns by index (equivilent to selecting values from a list): 

In [ ]:
trees.iloc[10:15]

If our dataframe index is arbitratry, this is not very useful.

We can assign a column as the index, then select by those values:

In [ ]:
trees_indexed = (
    trees
    .set_index('OBJECTID')
    .sort_index()
)

In [ ]:
trees_indexed.loc[3736475:3736480]

(note: use `.iloc[]` to select by row _number_, and `.loc[]` to select by index _label_)

Using a meaningful index like this is sometimes helpful. (we'll see soon that it is often useful for dates). But more often, you will want to filter data by different conditions.

### Filtering data

Using an inequality as a selector will return only the rows that match the condition:

In [ ]:
(
    trees
    [
        trees['DBH'].gt(10)
    ]
)

In [ ]:
(
    trees
    [
        trees['GenusSpecies'].eq('Acer - maple')
    ]
)

An alternative syntax is to use `.query()` with a query condition as a test string.

(I find the bracket syntax easier to read and write, but this is exactly equivalent; use what is easier for you)

In [ ]:
(
    trees
    .query('GenusSpecies == "Acer - maple"')
)

Multiple filter conditions can be applied together:

In [ ]:
(
    trees
    [
        (
            trees['DBH'].lt(10)
            |
            trees['StumpDiameter'].gt(5)
        )
        &
        trees['GenusSpecies'].eq('Pinus - pine')
    ]
)

A few other useful logical tests: 

`NaN` is never equal to any value. Instead use `.isna()` which returns `True` if the value is `NaN`, or `.notna()`, which returns `True` if the value is not `NaN`

In [ ]:
(
    trees
    [
        trees['StumpDiameter'].isna()
    ]
)

Use `.isin()` with a list of values to match to test whether each value matches any of these values:

In [ ]:
(
    trees
    [
        trees['TPCondition'].isin(['Excellent','Fair','Good'])
    ]
)

Use `~` as a logical `not`, meaning, the exact opposite of the filter condition. E.g. this returns the rows where the condition is _not_ in that list:

In [ ]:
(
    trees
    [
        ~
        trees['TPCondition'].isin(['Excellent','Fair','Good'])
    ]
)

Combining filters with summary statistics can show summaries for particular subsets of the data:

In [ ]:
(
    trees
    [
        trees['GenusSpecies'].isin(['Acer - maple','Acer platanoides - Norway maple'])
    ]
    ['DBH']
    .median()
)

## Groups

It is often useful to compare groups or subsets within the data

In [ ]:
(
    trees
    .groupby('TPCondition')
    ['DBH']
    .median()
)

`groupby()` creates subsets of the data for each value in the column (or columns) passed to it.

On its own, `.groupby()` doesn't produce anything useful: 

In [ ]:
trees.groupby('TPCondition')

...but each subsequent selection and summarization is applied to each subset, the results for all subsets are returned:

In [ ]:
(
    trees
    .groupby('TPCondition')
    ['DBH']
    .median()
)

use `.agg()` with a list of summary statistics to return different summarizations:

In [ ]:
(
    trees
    .groupby('TPCondition')
    ['DBH']
    .agg(['mean','median','max','count'])
)

Select multiple columns to aggregate:

In [ ]:
(
    trees
    .groupby('TPCondition')
    [['DBH','StumpDiameter']]
    .max()
)

Groups can be split using multiple columns:

In [ ]:
(
    trees
    .groupby(['TPStructure','TPCondition'])
    ['DBH']
    .mean()
)

`.unstack()` will pivot this into a wide-form table, which may be easier to view:

In [ ]:
(
    trees
    .groupby(['TPStructure','TPCondition'])
    ['DBH']
    .mean()
    .unstack()
)

Be cautious when subdividing data by multiple categories. You may cut the data too "thin" and wind up with categories that are too small to provide meaningful summaries. (We'll explore this more as we get into using statistics.)

In [ ]:
(
    trees
    .groupby(['TPStructure','TPCondition'])
    ['DBH']
    .count()
    .unstack()
)

Chaining together filters then summarization shows the summary just of the filtered data:

(You can try commenting-out the steps in this operation to see the resulting data at each step)

In [ ]:
(
    trees
    [
        trees['TPCondition'].eq('Good')
    ]
    .groupby(['GenusSpecies'])
    ['DBH']
    .median()
)

# Tasks

1. Select the subset of trees which have a `TPStructure` label of 'Retired'
2. How many days has a low temperature above 60 degrees and more than half an inch of rain?
3. Which tree species has the largest median diameter?
4. Which tree species has the most known stumps (counting only stumps with a diameter greater than 0 or a TPStructure of "Stump")?
5. What is the largest measured tree diameter of trees in each condition category?

In [ ]:
## Your code here:

Extra credit:
1. in fixing the weather data, you saw how we can assign a new column. Create a new column in the weather data called 'temperature span', and the high temperature minus the low temperature.
2. What is the mean temperature span?
3. How many days have a span greater than 15 degrees?